In [ ]:
# Torch Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# Data Loader Class
import os
import torch
import pandas as pd
import torchvision
import torchvision.io
from torchvision.io.image import read_image
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import cv2 as io

class Ham10000(Dataset):
    def __init__(self, csv_file, directory, transform, datasetname):
        self.annotations = pd.read_csv(f"{csv_file}")
        self.img_root_dir = 'dataverse_files'
        self.transform = transform
        self.datasetname = datasetname
        self.csv_file = csv_file
        self.directory = directory
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = ''
        img_path = os.path.join(self.directory, self.datasetname, self.annotations.iloc[idx, 1])
        img_path += ".jpg"

        x_img = io.imread(img_path)
        if self.transform:
            x_img = self.transform(x_img)

        y_label = torch.tensor(int(self.annotations.iloc[idx, 7]))

        return (x_img, y_label)




In [ ]:
# Train and Test
from sklearn.model_selection import KFold
import torch.nn as nn
import copy

# pass in model constructor
def balanced_train(model, dataset, device, num_epochs=10, loss_function=nn.CrossEntropyLoss()):
    train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])
    
    # Define data loaders for training and testing data
    trainloader = torch.utils.data.DataLoader(
                      train_set, 
                      batch_size=16, num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      test_set,
                      batch_size=16, num_workers=4)
    
    # Init the neural network
    network = copy.deepcopy(model)
    network.to(device)
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
    epoch_total_train_loss = list()
    epoch_train_acc = list()
    epoch_test_acc = []
    for epoch in range(0, num_epochs):
        print(f'Starting epoch {epoch+1}', '-', num_epochs)
        correct, total = 0, 0
        current_loss = 0.0
        
        network.train()
        train_loss = 0
        
        for i, data in enumerate(trainloader, 0):
            inputs, targets = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()
            outputs = network(inputs)

            _, predicted = torch.max(outputs.data, 1)

            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Print statistics
            train_loss+= loss.item()
            current_loss += loss.item()
            
        epoch_train_acc.append(100.0 * correct / total)
        epoch_total_train_loss.append(train_loss)
        
        # Evaluation
        network.eval()
        correct, total = 0, 0
        total_labels, total_preds = [],[]
        
        with torch.no_grad():
            for i, data in enumerate(testloader, 0):
                inputs, targets = data[0].to(device), data[1].to(device)
                outputs = network(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total_labels.extend(targets.data.cpu().detach().numpy())
                total_preds.extend(predicted.cpu().detach().numpy())
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
   
        epoch_test_acc.append(100.0 * correct / total)
        print("Acc {0}: {1}".format(epoch, 100.0 * correct / total))
    torch.save(network.state_dict(), "./balanced_{0}.pth".format(0))

    print("epoch_train_acc: ", epoch_train_acc)
    print("epoch_test_acc: ", epoch_test_acc)
    print("epoch_total_train_loss: ", epoch_total_train_loss)

In [ ]:
# Train and Test Balanced Model
csv_file = "./dataverse_files/HAM10000_metadata.csv"
directory = "./dataverse_files" 

transform = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomRotation(degrees=(13)),
        transforms.ColorJitter(brightness=.5, hue=.3),
        transforms.Normalize((0.1411, 0.0923, 0.5270), (0.3407, 0.3058, 0.2824))  
      ]
    )

batch_size = 32
datasetname = "HAM10000_images_off"
dataset = Ham10000(csv_file, directory, transform, datasetname)

resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features

resnet18.fc = nn.Sequential(
    nn.Linear(num_ftrs, 250),
    nn.Linear(250, 7)
)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet18 = resnet18.to(device)

# Manually Balance Dataset
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
df = pd.read_csv(csv_file)

X = df[['lesion_id','image_id','dx','dx_type','age','sex','localization']]
y = df[['label']]

# Undersample majority classes
rus = RandomUnderSampler(random_state=0,sampling_strategy={
    5:500,
    4:500,
    2:500,
    1:500
})

# Over sample Minority Classes
ros = RandomOverSampler(random_state=0)


X_resampled, y_resampled = rus.fit_resample(X,y)
X_resampled, y_resampled = ros.fit_resample(X_resampled,y_resampled)

df2 = pd.DataFrame(X_resampled)
df2 = pd.concat([df2, pd.DataFrame(y_resampled)],axis=1)
df2.columns = df.columns

dataset.annotations = df2

# Train Model
balanced_train(resnet18,dataset,device,num_epochs=2)

In [ ]:
# Process Results

fold1trainacc = [36.57142857142857, 44.892857142857146, 49.892857142857146, 52.285714285714285, 56.642857142857146, 56.82142857142857, 59.57142857142857, 61.82142857142857, 64.96428571428571, 67.60714285714286, 70.17857142857143, 72.82142857142857, 77.85714285714286, 78.92857142857143, 81.64285714285714, 83.89285714285714, 85.78571428571429, 87.60714285714286, 90.53571428571429, 91.96428571428571, 94.85714285714286, 93.82142857142857, 93.32142857142857, 96.78571428571429, 98.03571428571429, 96.10714285714286, 97.78571428571429, 97.64285714285714, 97.39285714285714, 96.42857142857143, 96.64285714285714, 96.32142857142857, 96.96428571428571, 98.42857142857143, 99.32142857142857, 99.21428571428571, 98.39285714285714, 98.14285714285714, 97.60714285714286, 95.75, 98.17857142857143, 97.78571428571429, 99.07142857142857, 97.60714285714286, 98.21428571428571, 99.07142857142857, 98.25, 98.53571428571429, 98.85714285714286, 98.57142857142857, 97.82142857142857, 97.75, 98.71428571428571, 97.60714285714286, 98.82142857142857, 98.14285714285714, 98.71428571428571, 97.96428571428571, 99.14285714285714, 99.0, 98.57142857142857, 99.67857142857143, 99.67857142857143, 99.57142857142857, 97.85714285714286, 97.71428571428571, 98.85714285714286, 99.21428571428571, 99.32142857142857, 99.14285714285714, 99.64285714285714, 99.75, 97.5, 97.64285714285714, 98.64285714285714, 99.32142857142857, 99.35714285714286, 99.35714285714286, 98.25, 98.71428571428571, 99.21428571428571, 99.28571428571429, 99.60714285714286, 99.67857142857143, 99.5, 99.07142857142857, 97.53571428571429, 97.67857142857143, 99.75, 99.42857142857143, 99.39285714285714, 99.64285714285714, 98.96428571428571, 99.03571428571429, 99.42857142857143, 99.78571428571429, 99.75, 99.92857142857143, 99.71428571428571, 99.78571428571429]
fold1testacc = [34.42857142857143, 43.142857142857146, 46.714285714285715, 31.571428571428573, 54.714285714285715, 53.0, 57.285714285714285, 61.857142857142854, 59.0, 68.0, 70.28571428571429, 63.0, 68.0, 57.57142857142857, 68.28571428571429, 67.0, 58.857142857142854, 70.57142857142857, 69.0, 68.28571428571429, 69.57142857142857, 68.57142857142857, 70.28571428571429, 71.28571428571429, 68.42857142857143, 71.42857142857143, 66.0, 73.85714285714286, 68.71428571428571, 70.42857142857143, 66.71428571428571, 70.57142857142857, 68.42857142857143, 71.57142857142857, 71.85714285714286, 72.14285714285714, 69.85714285714286, 74.14285714285714, 58.0, 69.57142857142857, 70.28571428571429, 69.57142857142857, 70.28571428571429, 65.42857142857143, 72.42857142857143, 70.0, 69.85714285714286, 71.85714285714286, 71.14285714285714, 69.0, 70.71428571428571, 70.14285714285714, 69.57142857142857, 72.0, 73.42857142857143, 71.85714285714286, 70.57142857142857, 64.85714285714286, 74.14285714285714, 70.42857142857143, 72.85714285714286, 68.14285714285714, 71.42857142857143, 72.57142857142857, 63.714285714285715, 72.0, 70.71428571428571, 70.71428571428571, 75.0, 74.14285714285714, 73.0, 73.71428571428571, 68.71428571428571, 71.14285714285714, 72.71428571428571, 73.28571428571429, 71.28571428571429, 72.0, 69.28571428571429, 72.57142857142857, 70.14285714285714, 71.71428571428571, 73.28571428571429, 68.85714285714286, 72.85714285714286, 70.0, 65.57142857142857, 72.14285714285714, 75.14285714285714, 71.57142857142857, 71.0, 73.0, 65.57142857142857, 69.28571428571429, 70.0, 71.0, 70.85714285714286, 71.0, 70.14285714285714, 72.14285714285714]
total1_train_loss =  [288.64105808734894, 253.0054950118065, 230.06339144706726, 220.27727037668228, 202.99596786499023, 204.66004538536072, 191.30695521831512, 182.33097791671753, 161.43051770329475, 151.45955961942673, 141.0156622827053, 128.22024568915367, 104.50479678809643, 100.10595579445362, 85.35560727864504, 74.70597901195288, 70.40183169394732, 59.451678020879626, 47.28546370565891, 39.913006384857, 26.9441495006904, 29.9001996552106, 32.57642619358376, 16.768864060286433, 11.598927564627957, 22.307509062229656, 11.953715903189732, 11.587976902315859, 16.992744474700885, 16.706920441181865, 19.042762525030412, 21.459831377258524, 15.870849806640763, 8.30543706878234, 3.678805995732546, 4.188055371429073, 9.62781457836536, 8.595020865024708, 14.270928730307787, 23.42553110120207, 9.170630173830432, 11.643290205698577, 5.7119786360053695, 13.015208919285215, 10.241644085006556, 4.802134670950181, 9.108510731373826, 6.879127049025556, 6.320140517276741, 8.7901391467949, 11.552828445346677, 11.913466515106848, 7.086564750250545, 14.080108179825402, 6.336836043867152, 10.673922009530088, 6.7063351551114465, 11.821447665841333, 5.19582011264356, 6.136856860926855, 6.830799220932022, 2.168359954526295, 1.7766993201764762, 2.6664769003562014, 13.039119901642607, 13.148054234756273, 5.83832527370032, 3.62253315669534, 3.888923205438914, 4.429166579351204, 2.019957647564752, 1.6141590105331716, 16.30771381085242, 13.234555035676749, 7.53928292854107, 4.188204947735358, 2.713339406585874, 2.912919460013427, 11.047591493583695, 6.512957315571839, 3.8386384845234716, 2.804182943824685, 2.5661721667147503, 1.254026689533589, 2.3798158056679313, 3.867416982235767, 17.090785690572375, 15.738884262249485, 1.3429044939803134, 2.007777934425576, 4.182704176790594, 2.337307177499042, 6.801806309172719, 4.836476223355021, 3.3000150544694407, 1.3234703602585682, 1.399501429144948, 0.3415576756437133, 1.1104098317110314, 1.2451840455609329]


total_train_loss_append = [total1_train_loss]
total_train_loss = []
for i in range(len(total1_train_loss)):
  total_loss = []
  for j in range(len(total_train_loss_append)):
    total_loss.append(total_train_loss_append[j][i])
  total_train_loss.append(sum(total_loss) / len(total_loss))
print(total_train_loss)


total_train_acc_append = [fold1trainacc]

total_train_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_train_acc_append)):
    total_acc.append(total_train_acc_append[j][i])
  total_train_acc.append(sum(total_acc) / len(total_acc))
print(total_train_acc)


total_test_acc_append = [fold1testacc]

total_test_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_test_acc_append)):
    total_acc.append(total_test_acc_append[j][i])
  total_test_acc.append(sum(total_acc) / len(total_acc))
print(total_test_acc)

In [ ]:
# Loss Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import pandas as pd

epochs = list(range(0,len(total_train_loss)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Loss': total_train_loss,
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Loss',data=data).set(title="Train Loss")

In [ ]:
# Accuracy Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

epochs = list(range(0,len(total_train_acc)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Training Accuracy': total_train_acc,
    'Testing Accuracy': total_test_acc
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Accuracy',hue='variable',data=pd.melt(data, ['Epoch'],value_name="Accuracy")).set(title="Train vs Test Accuracy")
